# Portfolio Optimization Using Modern Portfolio Theory (MPT)

This notebook uses forecasted and historical returns for TSLA, BND, and SPY to construct optimal portfolios using MPT.
- TSLA: Expected return from forecast (Task 2)
- BND & SPY: Historical annualized returns
- Covariance matrix from historical returns
- Efficient Frontier, Maximum Sharpe, Minimum Volatility portfolios
    

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

np.random.seed(42)


## 1. Load Data and Calculate Returns

In [ ]:
tickers = ['TSLA', 'BND', 'SPY']
data = yf.download(tickers, start='2018-01-01', auto_adjust=True)['Close']
returns = data.pct_change().dropna()
annual_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252
annual_returns

## 2. Set Expected Returns
- Use forecasted return for TSLA (replace below with your model's forecasted annual return)
- Use historical annualized returns for BND, SPY

In [ ]:
# Replace with your best model's annualized forecast for TSLA
tsla_expected = 0.18  # Example: 18% forecasted annual return
expected_returns = annual_returns.copy()
expected_returns['TSLA'] = tsla_expected
expected_returns

## 3. Efficient Frontier Simulation

In [ ]:
def portfolio_performance(weights, mean_returns, cov_matrix, rf=0.02):
    returns = np.dot(weights, mean_returns)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = (returns - rf) / volatility
    return returns, volatility, sharpe

def simulate_portfolios(n, mean_returns, cov_matrix, rf=0.02):
    results = np.zeros((n, 3 + len(mean_returns)))
    for i in range(n):
        weights = np.random.dirichlet(np.ones(len(mean_returns)), size=1)[0]
        ret, vol, sharpe = portfolio_performance(weights, mean_returns, cov_matrix, rf)
        results[i, :3] = [ret, vol, sharpe]
        results[i, 3:] = weights
    return results

sim_results = simulate_portfolios(10000, expected_returns, cov_matrix)
cols = ['Return', 'Volatility', 'Sharpe'] + tickers
sim_df = pd.DataFrame(sim_results, columns=cols)
sim_df.head()

## 4. Optimize for Maximum Sharpe and Minimum Volatility

In [ ]:
def min_func_sharpe(weights, mean_returns, cov_matrix, rf=0.02):
    return -portfolio_performance(weights, mean_returns, cov_matrix, rf)[2]
def min_func_volatility(weights, mean_returns, cov_matrix):
    return portfolio_performance(weights, mean_returns, cov_matrix)[1]
num_assets = len(tickers)
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(num_assets))
init_guess = num_assets * [1. / num_assets,]
max_sharpe = minimize(min_func_sharpe, init_guess, args=(expected_returns, cov_matrix), method='SLSQP', bounds=bounds, constraints=constraints)
min_vol = minimize(min_func_volatility, init_guess, args=(expected_returns, cov_matrix), method='SLSQP', bounds=bounds, constraints=constraints)
max_sharpe_w = max_sharpe.x
min_vol_w = min_vol.x
max_sharpe_perf = portfolio_performance(max_sharpe_w, expected_returns, cov_matrix)
min_vol_perf = portfolio_performance(min_vol_w, expected_returns, cov_matrix)
max_sharpe_w, max_sharpe_perf, min_vol_w, min_vol_perf

## 5. Plot Efficient Frontier and Key Portfolios

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(sim_df['Volatility'], sim_df['Return'], c=sim_df['Sharpe'], cmap='viridis', alpha=0.7)
plt.colorbar(label='Sharpe Ratio')
plt.scatter(max_sharpe_perf[1], max_sharpe_perf[0], marker='*', color='r', s=300, label='Max Sharpe')
plt.scatter(min_vol_perf[1], min_vol_perf[0], marker='X', color='b', s=200, label='Min Volatility')
plt.xlabel('Volatility (Risk)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier: TSLA, BND, SPY')
plt.legend()
plt.show()

## 6. Final Portfolio Recommendation and Summary

- **Max Sharpe Portfolio Weights:**
    - TSLA: {max_sharpe_w[0]:.2f}
    - BND: {max_sharpe_w[1]:.2f}
    - SPY: {max_sharpe_w[2]:.2f}
- **Expected Annual Return:** {max_sharpe_perf[0]:.2%}
- **Annual Volatility:** {max_sharpe_perf[1]:.2%}
- **Sharpe Ratio:** {max_sharpe_perf[2]:.2f}

- **Minimum Volatility Portfolio Weights:**
    - TSLA: {min_vol_w[0]:.2f}
    - BND: {min_vol_w[1]:.2f}
    - SPY: {min_vol_w[2]:.2f}
- **Expected Annual Return:** {min_vol_perf[0]:.2%}
- **Annual Volatility:** {min_vol_perf[1]:.2%}
- **Sharpe Ratio:** {min_vol_perf[2]:.2f}

**Recommendation:**
Choose the portfolio that aligns with your risk tolerance and investment goals. For maximum risk-adjusted return, select the Max Sharpe portfolio. For lowest risk, select the Min Volatility portfolio.
